In [5]:
import pandas as pd

In [9]:
!ls

__init__.py                      data_fcns.py
__pycache__                      dev
comic_recs.py                    geckodriver.log
comic_scraper.py                 keys.py
create_lists_for_dropdowns.ipynb webapp


In [14]:
path = './dev/support_data/comics.json'

In [15]:
comics = pd.read_json(path, lines=True)

In [16]:
comic_ids = comics['comic_id'].tolist()

In [17]:
comic_titles = comics['comic_title'].tolist()

In [18]:
comic_ids[:4]

[1, 2, 3, 4]

In [19]:
comic_titles[:10]

['0Secret Wars (Marvel)',
 '100 Bullets Brother Lono (DC)',
 '100 Penny Press Locke & Key (IDW)',
 '100 Penny Press Star Trek (IDW)',
 '100 Penny Press Thunder Agent (IDW)',
 '100 Penny Press Tmnt (IDW)',
 '100th Anniversary Special (Marvel)',
 '12 Reasons To Die (Other)',
 '13 Coins (Other)',
 '13th Artifact One Sho (Topcow)']

In [20]:
!pwd

/Users/werlindo/Dropbox/flatiron/capstone/comics_rx/comrx


In [21]:
import pandas as pd
import requests
import random
import time
import os

# Data storage
from sqlalchemy import create_engine # SQL helper
import keys     # Custom keys lib
import psycopg2 as psql #PostgreSQL DBs

In [22]:
# Define path to secret
secret_path_aws = os.path.join(os.environ['HOME'], '.secret', 
                           'aws_ps_flatiron.json')
secret_path_aws

'/Users/werlindo/.secret/aws_ps_flatiron.json'

In [23]:
aws_keys = keys.get_keys(secret_path_aws)
user = aws_keys['user']
ps = aws_keys['password']
host = aws_keys['host']
db = aws_keys['db_name']

aws_ps_engine = ('postgresql://' + user + ':' + ps + '@' + host + '/' + db)

# Setup PSQL connection
conn = psql.connect(
    database=db,
    user=user,
    password=ps,
    host=host,
    port='5432'
)

In [24]:
# Instantiate cursor
cur = conn.cursor()

In [25]:
#  Count records.
query = """
    SELECT ct.*, c.comic_id from comic_trans ct
    inner join comics c on ct.comic_title = c.comic_title;
"""

In [26]:
# Execute the query
cur.execute(query)

In [27]:
# Check results
temp_df = pd.DataFrame(cur.fetchall())
temp_df.columns = [col.name for col in cur.description]

temp_df.head(3)

,index,publisher,item_id,title_and_num,qty_sold,date_sold,account_num,comic_title,comic_id
0,2,Amaze Ink Slave Labor Graphics,DCD151935,Filler Bunny #2,1,2011-08-14 18:01:03,00174,Filler Bunny (SLG),2291
1,3,Amaze Ink Slave Labor Graphics,DCD341726,Gargoyles #6,1,2012-06-22 14:11:37,00593,Gargoyles (SLG),2445
2,4,Amaze Ink Slave Labor Graphics,DCD416182,Royal Historian of Oz #1,1,2010-07-21 14:03:07,00226,Royal Historian of Oz (SLG),5178


In [28]:
qtys = (temp_df.groupby(['comic_id','comic_title'], as_index=False)
        .qty_sold.sum().sort_values(by=['qty_sold'], ascending=False))

In [29]:
top_100 = qtys.head(100)

In [30]:
top_100 = (top_100.sort_values(by=['comic_title'], ascending=True).
                    loc[:,['comic_id', 'comic_title']])

In [33]:
top_100_dict = top_100.to_dict('list')

In [34]:
top_100_dict

{'comic_id': [81,
  198,
  224,
  278,
  319,
  388,
  433,
  455,
  552,
  610,
  619,
  648,
  578,
  691,
  804,
  829,
  838,
  960,
  992,
  1077,
  1143,
  1214,
  1345,
  1492,
  1560,
  1709,
  1801,
  1807,
  1906,
  1963,
  2041,
  2171,
  2217,
  2228,
  2281,
  2318,
  2680,
  2763,
  2783,
  2779,
  2842,
  2939,
  2950,
  2975,
  3386,
  3388,
  3390,
  3568,
  3630,
  3643,
  3700,
  3820,
  3982,
  4028,
  4248,
  4316,
  4351,
  4455,
  4503,
  4632,
  4677,
  4701,
  4919,
  5196,
  5212,
  5238,
  5284,
  5286,
  5318,
  5348,
  5506,
  5600,
  5659,
  5836,
  5882,
  6019,
  6036,
  6052,
  6069,
  6099,
  6105,
  6175,
  6243,
  6262,
  6260,
  6667,
  6681,
  6684,
  6692,
  6694,
  6722,
  6761,
  6824,
  6937,
  6987,
  7021,
  7079,
  7126,
  7125,
  7193],
 'comic_title': ['Action Comics (DC)',
  'All New X-Men (Marvel)',
  'Amazing Spider-Man (Marvel)',
  'Animal Man (DC)',
  'Aquaman (DC)',
  'Astonishing X-Men (Marvel)',
  'Avengers (2010) (Marvel)',
  'Ave

In [40]:
zaz = pd.DataFrame(top_100_dict)

In [41]:
zaz

,comic_id,comic_title
0,81,Action Comics (DC)
1,198,All New X-Men (Marvel)
2,224,Amazing Spider-Man (Marvel)
3,278,Animal Man (DC)
4,319,Aquaman (DC)
5,388,Astonishing X-Men (Marvel)
6,433,Avengers (2010) (Marvel)
7,455,Avengers (Marvel)
8,552,Batgirl (DC)
9,610,Batman (DC)


### Export back out to csv

In [56]:
top_100.to_csv('./webapp/templates/dev_files/top_100_comics.csv', 
               index=False)

### Make lists

In [45]:
coms = pd.read_csv('./webapp/templates/dev_files/top_100_comics.csv')

In [46]:
coms.head()

,comic_id,comic_title
0,81,Action Comics (DC)
1,198,All New X-Men (Marvel)
2,224,Amazing Spider-Man (Marvel)
3,278,Animal Man (DC)
4,319,Aquaman (DC)


In [47]:
ids = coms['comic_id'].tolist()

In [48]:
titles = coms['comic_title'].tolist()

In [49]:
ids[:10]

[81, 198, 224, 278, 319, 388, 433, 455, 552, 610]

In [43]:
titles[:10]

NameError: name 'titles' is not defined

In [50]:
ddd = ("dfdfdfd" +
        "dfdfdfdf" +
        "dfdfdfdfdf" )
ddd


'dfdfdfddfdfdfdfdfdfdfdfdf'

In [52]:
zzz = coms.head(5)['comic_title']

In [53]:
zzz

0             Action Comics (DC)
1         All New X-Men (Marvel)
2    Amazing Spider-Man (Marvel)
3                Animal Man (DC)
4                   Aquaman (DC)
Name: comic_title, dtype: object

In [54]:
type(zzz)

pandas.core.series.Series

In [56]:
for i, row in zzz.iteritems():
    print(row)

Action Comics (DC)
All New X-Men (Marvel)
Amazing Spider-Man (Marvel)
Animal Man (DC)
Aquaman (DC)


In [57]:
import flask

In [58]:
from flask import Flask, request, render_template, jsonify, session, redirect, make_response

In [70]:
www = [{'comic_id': 81, 'comic_title': 'Action Comics (DC)'}, {'comic_id': 198, 'comic_title': 'All New X-Men (Marvel)'}, {'comic_id': 224, 'comic_title': 'Amazing Spider-Man (Marvel)'}, {'comic_id': 278, 'comic_title': 'Animal Man (DC)'}, {'comic_id': 319, 'comic_title': 'Aquaman (DC)'}]

In [71]:
www

[{'comic_id': 81, 'comic_title': 'Action Comics (DC)'},
 {'comic_id': 198, 'comic_title': 'All New X-Men (Marvel)'},
 {'comic_id': 224, 'comic_title': 'Amazing Spider-Man (Marvel)'},
 {'comic_id': 278, 'comic_title': 'Animal Man (DC)'},
 {'comic_id': 319, 'comic_title': 'Aquaman (DC)'}]

In [72]:
type(www)

list

In [73]:

www[0]

{'comic_id': 81, 'comic_title': 'Action Comics (DC)'}

In [74]:
www[0]['comic_id']

81